This python script finds the code of the building closest to the Cristo Redentor using the data of this database : https://sites.research.google/open-buildings

In [1]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
# Creation of the point at the coordinate of the Cristo Redentor statue
STATUE_POINT = Point(-43.2075000,-22.9027800)

Craping GeoJSON data containing tiles information

In [2]:
URL = "https://sites.research.google/open-buildings/tiles.geojson"
response = requests.get(URL)
json_resp = response.json()

Finding the tile url of Rio considering that the statue is in Rio.

In [3]:
for feature in json_resp['features']:
    RIO_TILE_POLY = Polygon(feature["geometry"]["coordinates"][0])
    if RIO_TILE_POLY.contains(STATUE_POINT):
        RioFeature = feature
tileURL = RioFeature["properties"]["tile_url"]
# Map on the web site show Rio tile id is 009  
print(RioFeature["properties"]["tile_id"] == '009')


True


Checking if the statue point is in the polygone of Rio

In [4]:
STATUE_POINT = Point(-43.2075000,-22.9027800)
RIO_TILE_POLY = Polygon([ [ -45.0, -25.9260509 ], [ -39.9812347, -27.7802732 ], [ -39.9812347, -22.7359095 ], [ -45.0, -21.1414374 ], [ -45.0, -25.9260509 ] ] )
print(RIO_TILE_POLY.contains(STATUE_POINT))

True


Creation of the GeoDataFrame of Rio buildings from the tile url

In [5]:
print('tile url : ',tileURL)
df_Rio_Buildings = pd.read_csv(tileURL, compression='gzip')
gdf_Rio_Buildings = gpd.GeoDataFrame(
    df_Rio_Buildings, geometry=gpd.points_from_xy(df_Rio_Buildings.longitude, df_Rio_Buildings.latitude), crs="EPSG:4326"
)

tile url :  https://storage.googleapis.com/open-buildings-data/v3/polygons_s2_level_4_gzip/009_buildings.csv.gz


Finding the closest building 

In [6]:
gdf_Rio_Buildings['distance_statue'] = gdf_Rio_Buildings['geometry'].distance(STATUE_POINT)
closest_building = gdf_Rio_Buildings.loc[gdf_Rio_Buildings['distance_statue'].idxmin()]

C:\Users\Maxime\AppData\Local\Temp\ipykernel_24160\3307619193.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_Rio_Buildings['distance_statue'] = gdf_Rio_Buildings['geometry'].distance(STATUE_POINT)


Code number of the closest building to the Cristo Redentor statue

In [ ]:
print(closest_building["full_plus_code"])

589R3QWR+WX9W
